Notebook used mainly for the Data Science Capstone project

In [1]:
# Import the Pandas package
import pandas as pd

# Import the NumPy package
import numpy as np

In [2]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Part 1

### Get HTML data from wikipedia

In [7]:
Canada_M = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Scrape PostalCode, Borough, Neighborhood from HTML using BeautifulSoup

In [8]:
# soup = BeautifulSoup(open("Canada_M.htm"), "lxml")
soup = BeautifulSoup(Canada_M, "lxml")

In [11]:
# Create a dataframe makes up of three columns: PostalCode, Borough, and Neighborhood
PostalCode = []
Borough = []
Neighborhood = []

for data in soup.tbody.find_all('tr')[1:]:
    PostalCode.append(data.find('td').text)
    Borough.append(data.find_all('td')[1].text)
    Neighborhood.append(data.find_all('td')[2].text[:-1])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [13]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

data = {'PostalCode': PostalCode,
        'Borough': Borough,
        'Neighborhood': Neighborhood}
df = pd.DataFrame(data)
# Drop rows where Borough is 'Not assigned', then reset index
df = df[df.Borough != 'Not assigned\n'].reset_index().drop('index', axis = 1)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


### More than one neighborhood can exist in one postal code area, these two rows will be combined into one row with the neighborhoods

In [14]:
i = 1
while(i < len(df)):
    if df['PostalCode'].iloc[i] == df['PostalCode'].iloc[i - 1]:
        df.at[i - 1, 'Neighborhood'] = df.Neighborhood.iloc[i - 1] +', ' + df.Neighborhood.iloc[i]
        df.drop(index = i, inplace = True)
        df = df.reset_index().drop('index', axis = 1)
    else:
        i += 1

In [20]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government
5,M9A\n,Etobicoke\n,Islington Avenue
6,M1B\n,Scarborough\n,Malvern / Rouge
7,M3B\n,North York\n,Don Mills
8,M4B\n,East York\n,Parkview Hill / Woodbine Gardens
9,M5B\n,Downtown Toronto\n,Garden District / Ryerson


### Print the number of rows of your dataframe by using .shape

In [21]:
df.shape

(103, 3)